In [ ]:
import pandas as pd
import os
import gzip
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CF_Resources

/content/drive/MyDrive/CF_Resources


In [ ]:
# specify the path to your json.gz file
file_path = "AMAZON_FASHION.json.gz"

# # open the gzip file in read binary mode
# with gzip.open(file_path, 'rb') as f:
#     # read the contents of the gzip file and decode it as utf-8
#     file_contents = f.read().decode('utf-8')
#     # parse the JSON string into a Python dictionary
#     data = json.loads(file_contents)

# # now you can access the data as a dictionary
# type(data)

In [ ]:
data = []

with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            obj = json.loads(line)
            data.append(obj)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2.0,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3,NaN,NaN
2,4.0,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2.0,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3.0,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [ ]:
df = df.drop(columns=[ 'vote', 'style', 'image', 'reviewTime'])

In [ ]:
df.shape

(883636, 8)

In [ ]:
df.head()

,overall,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime
0,5.0,True,A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200
1,2.0,True,A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4.0,False,A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2.0,True,A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400
4,3.0,False,A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200


In [ ]:
df['reviewerID'] = pd.factorize(df['reviewerID'])[0]
df['asin'] = pd.factorize(df['asin'])[0]
df.dropna(subset=['reviewText'], inplace=True)
df.head()

,overall,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime
0,5.0,True,0,0,Tracy,Exactly what I needed.,perfect replacements!!,1413763200
1,2.0,True,1,0,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4.0,False,2,0,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2.0,True,3,0,Jodi Stoner,too tiny an opening,Two Stars,1408838400
4,3.0,False,4,0,Alexander D.,Okay,Three Stars,1406419200


# New section

In [ ]:
def processAmazonReviews(reviews_df):
    # # Load the review data
    # reviews_df = pd.read_json(os.path.join(path, "reviews_Amazon_Fashion_5.json.gz"), lines=True)

    # Create a dictionary of word to word_id
    word_id_dict = {}
    with open("WordDict.out", "w") as f:
        idx = 0
        for review in reviews_df["reviewText"]:
            if(type(review)== str):
              review_words = review.split()
              for word in review_words:
                  if word not in word_id_dict:
                      word_id_dict[word] = idx
                      idx += 1
                      f.write(word + "\t" + str(word_id_dict[word]) + "\n")


    # Create user review document and item review document files
    with open("UserReviews.out", "w") as f_user, open("ItemReviews.out", "w") as f_item:
        for index, row in reviews_df.iterrows():
            user_review = str(row["reviewerID"]) + "\t" + " ".join([str(word_id_dict[word]) for word in row["reviewText"].split()]) + "\n"
            item_review = str(row["asin"]) + "\t" + " ".join([str(word_id_dict[word]) for word in row["reviewText"].split()]) + "\n"
            f_user.write(user_review)
            f_item.write(item_review)

    # Create training, validation, and testing rating files
    ratings_df = reviews_df[["reviewerID", "asin", "overall", "unixReviewTime"]]

    #ratings_df = ratings_df.sort_values(by="unixReviewTime")
    total_size = len(reviews_df)
    train_size = int(total_size * 0.8)
    val_size = int(total_size * 0.1)

    with open("TrainInteraction.out", "w") as f_train, \
         open("ValInteraction.out", "w") as f_val, \
         open("TestInteraction.out", "w") as f_test:
        for i, row in ratings_df.iterrows():

            if i < train_size:
                f_train.write(str(int(row["reviewerID"])) + "\t" + str(int(row["asin"])) + "\t" + str(row["overall"]) + "\t" + str(int(row["unixReviewTime"])) + "\n")
            elif i < train_size + val_size:
                f_val.write(str(int(row["reviewerID"])) + "\t" + str(int(row["asin"])) + "\t" + str(row["overall"]) + "\t" + str(int(row["unixReviewTime"])) + "\n")
            else:
                f_test.write(str(int(row["reviewerID"])) + "\t" + str(int(row["asin"])) + "\t" + str(row["overall"]) + "\t" + str(int(row["unixReviewTime"])) + "\n")

    print("Processing Amazon reviews dataset finished.")


In [ ]:
%cd /content
processAmazonReviews(df)

/content
Processing Amazon reviews dataset finished.


In [ ]:
! python Carl.py